In [ ]:
import wave
import numpy
import matplotlib.pyplot as plt 
import pydub
from playsound import playsound
from pydub import AudioSegment
from IPython.display import Audio
import sounddevice as sd
import time
import os
from tkinter import filedialog
from tkinter import *
import soundfile as sf
from pydub.utils import which , mediainfo
import ffmpeg

# <font color="black"><u>Exercice 1</u> :</font>

In [ ]:
#Q1 : Load an audio file

def open_file ():
    window = Tk()
    window.withdraw()
    file = filedialog.askopenfilename()
    return file

In [ ]:
#Q2 : Extract parameters (tracks, channel...)

def get_para(audio):
    (nc,sw,fr,nfr,ct,cn)=audio.getparams()
    return (nc,sw,fr,nfr,ct,cn)

def channels (nc):
    if(nc==1):
        print("Mono")
    elif(nc==2):
        print("Streo")

In [ ]:
aud=wave.open(open_file())
(nc,sw,fr,nfr,ct,cn)=get_para(aud)
print(nc,sw,fr,nfr,ct,cn)

In [ ]:
channels(nc)

In [ ]:
#Q3: Extract each audio track in a vector

def ext(audio):
    data= audio.readframes(-1)
    data_hex= numpy.frombuffer(data,dtype=type(audio.getsampwidth()))
    return data_hex

def split(data_hex):
    vect1=[data_hex[i] for i in range(len(data_hex)) if(i%2==0)]
    vect2=[data_hex[i] for i in range(len(data_hex)) if(i%2!=0)]
    vect=[vect1,vect2]
    return vect

In [ ]:
data_hex=ext(aud)
print(data_hex)
vect1,vect2=split(data_hex)
print(vect1,vect2)

In [ ]:
#Q4 : Graphical representation

def trace_1(vect,nc):
    if nc==1:
        plt.figure(figsize=(20,15))
        plt.plot(vect)
        plt.savefig("plot1.png",bbox_inches ="tight")
    else:
        for i in range(1,nc+1):
            plt.figure(figsize=(20,15))
            plt.subplot(3,1,i)
            plt.plot(vect[i-1])
            name="plot"+str(i+1)+".png"
            plt.savefig(name,bbox_inches ="tight")

In [ ]:
vect=[vect1,vect2]
trace_1(vect,2)

In [ ]:
#Q5 : Extract a part of a track

def partie(k1,k2,vect):
    return vect[k1:k2]

In [ ]:
partie(1000,2000,vect1)

In [ ]:
#Q6 : Represent the extracted graphically as a function of the sampling period

def trace_2(ext_vect):
    #the sampling period
    temps= 1/fr
    t=[i*temps for i in range(0,len(vect2))]
    plt.plot(t,vect2)

In [ ]:
trace_2(partie(1000,2000,vect1))

In [ ]:
#Q7 : Listen to the signals

def listen(vect1,vect2,fr):
    sd.play(vect1,fr/2)
    time.sleep(2)
    sd.play(vect2,fr/2)

In [ ]:
listen(vect1,vect2,fr)

In [ ]:
#Q8 : Generate .wav files for each part

def save(vect,nc,sw,fr,nfr,ct,cn):
    if nc==1:
        audio=wave.open("extrait.wav","w")
        audio.setparams(nc,sw,fr,nfr,ct,cn)
        audio.writeframes(bytes(numpy.array(vect)))
    else :    
        (data,fr)=sf.read(open_file())
        data_r=[[0,0] for i in range(len(data))]
        data_l=[[0,0] for i in range(len(data))]
        for i in range(len(data)):
            data_r[i][0]=0
            data_r[i][1]=data[i][1]
            data_l[i][0]=data[i][0]
            data_l[i][1]=0
        sf.write("extrait1.wav",data_r,fr,"PCM_16")
        sf.write("extrait2.wav",data_l,fr,"PCM_16")

# <font color="black"><u>Exercice 2</u> :</font>

In [ ]:
#Q1 : Get file size

#we can use os lib :os.path.getsize(open_file())/1024

def size(nc,fr,nfr,sw):
    t=float(nfr/fr)
    taille=(nc*fr*t*sw)/1024
    return taille

In [ ]:
print(size(nc,fr,nfr,sw),"ko")

In [ ]:
#Q2 : Increasing/decreasing the pitch of the sound 

def vol(w,db):                          # w est audio de type Audiosegment et db est une valeur (-\+ )
    w=w+db
    w.export("file.wav", "wav")

In [ ]:
#Q3_A : Subsample a signal at different frequencies

def sous_ech(name_wav,f):
    aud1=wave.open(open_file())
    (nc,sw,fr,nfr,ct,cn)=get_para(aud1)
    data_hex1=ext(aud1)
    audio1=wave.open(name_wav,"w")
    audio1.setparams((nc,sw,fr//f,nfr,ct,cn))
    audio1.writeframes(bytes(numpy.array(data_hex1[0::f])))
    audio1.close()

In [ ]:
sous_ech("file.wav",2)

In [ ]:
#Q3_B : Trace the signal and listen to it

def compare_trace():
    org=wave.open(open_file())
    data_org= org.readframes(-1)
    data_hex_org= numpy.frombuffer(data_org,numpy.int16)
    cnvrt=wave.open(open_file())
    data_cnvrt= cnvrt.readframes(-1)
    data_hex_cnvrt= numpy.frombuffer(data_cnvrt,numpy.int16)
    vect=[data_hex_org,data_hex_cnvrt]
    trace_1(vect,2)

In [ ]:
compare_trace()

In [ ]:
#Q3_C : Get the max freq and calculate the new size

def fre_max():
    w=wave.open(open_file())
    (nc1,sw1,fr1,nfr1,ct1,cn1)=get_para(w)
    print("Fe Max : ",fre//2)
    print(size(nc1,fr1,nfr1,sw1),"ko")

In [ ]:
#Q4_A : Convert audio files to different bits (8,24...)

#You can use the soundfile library function to find the subtypes of a formatCalculate the compression ratio:sf.available_subtypes('ogg')

def convert(name_wave,typ):
    (data,fr)=sf.read(open_file())
    sf.write(name_wave,data,fr,typ)

In [ ]:
convert("file.wav","PCM_24")

In [ ]:
#Q4_B : #Compare the original with re-quantized

def compare_listen(n):                  # n est le nombre d'audio convertir
    for i in range(n):
        playsound(open_file())

In [ ]:
#Q4_C : Compare the max/min values between the two

def max_min():
    audio1=wave.open(open_file())
    a1= audio1.readframes(-1)
    b1= numpy.frombuffer(a1,numpy.int16)
    audio2=wave.open(open_file())
    a2= audio2.readframes(-1)
    b2= numpy.frombuffer(a2,numpy.int8)
    print("Valeur min d'original : ",min(b1),"Valeur min codable : ",min(b2))
    print("Valeur max d'original : ".max(b1),"Valeur max codable : ",max(b2))

In [ ]:
#Q4_D : Calculate the compression ratio

def facteur(n):
    liste=[]                                             # n est le nombre d'audio convertir
    for i in range(n):
        t1=os.path.getsize(open_file())/1024
        t2=os.path.getsize(open_file())/1024
        liste.append(t1/t2)
    return liste

# <font color="black"><u>Exercice 3</u> :</font>

In [ ]:
#Q_1_2_3_4 : Compress audio file 

def compress(ext,formate):
    AudioSegment.converter = which("ffmpeg")
    file_path=open_file()
    file=os.path.splitext(file_path)[0]+ ext
    aud1= AudioSegment.from_file(file_path)
    aud1.export(file,format=formate)

In [ ]:
compress(".mp3","mp3")

In [ ]:
#Q5 : Calculate the compression ratio

def taux():
    t1=os.path.getsize(open_file())/1024
    t2=os.path.getsize(open_file())/1024
    print(t1/t2)